In [3]:
import pandas as pd

In [4]:
min_grant_perc = 0.30 # 30% of per capita entitlement

# Victoria

In [5]:
vic_councils = pd.read_excel("../Data/FA Grants Data Tables.xlsx", sheet_name="Victoria")

In [6]:
vic_councils

,Council,Standardised Expenditure,Standardised Revenue,Funding Gap,Grant unadjusted,ERP,Unadjusted Per Capita,Deficit,Minimum Grant (PC basis),Raw Allocation,MGC,New Grant,New Grant Per Capita,Change %,Per Capita Change,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Alpine S,33015659,12720962,20294697,3407971,13578,250.992120,1,173771.244,3.738298e+06,0,3.601740e+06,265.262948,5.69,14.27,NaN,NaN,NaN
1,Ararat RC,36751217,8724183,28027034,4706418,11683,402.843277,1,149519.034,5.162601e+06,0,4.974014e+06,425.747972,5.69,22.90,NaN,NaN,NaN
2,Ballarat C,166858908,78156611,88702296,14895265,118137,126.084673,1,1511917.326,1.633903e+07,0,1.574217e+07,133.253538,5.69,7.17,NaN,NaN,NaN
3,Banyule C,150894932,136357102,14537830,3317393,129602,25.596773,1,1658646.396,2.677879e+06,0,2.580058e+06,19.907545,-22.23,-5.69,NaN,Total Grant to State,5.807909e+08
4,Bass Coast S,93645282,52867488,40777794,6847580,42729,160.256032,1,546845.742,7.511300e+06,0,7.236916e+06,169.367791,5.69,9.11,NaN,Per Capita,8.532000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Wodonga C,61963227,25132421,36830806,6184785,44276,139.687077,1,566644.248,6.784262e+06,0,6.536437e+06,147.629337,5.69,7.94,NaN,NaN,NaN
75,Wyndham C,352482859,206999459,145483400,24430188,324087,75.381573,1,4147665.426,2.679815e+07,0,2.581923e+07,79.667586,5.69,4.29,NaN,NaN,NaN
76,Yarra C,118797442,177246432,-58448991,2494354,97448,25.596770,0,1247139.504,0.000000e+00,1,1.247140e+06,12.798000,-50.00,-12.80,NaN,NaN,NaN
77,Yarra Ranges S,208326919,127797272,80529647,13522879,158694,85.213549,1,2030965.812,1.483362e+07,0,1.429176e+07,90.058584,5.69,4.85,NaN,NaN,NaN
